In [14]:
import json
from bs4 import BeautifulSoup
import requests
import os

def code(soup):
    # Find all doctor info boxes
    doctor_info_boxes = soup.find_all('div', class_='doctor-info-box-in')

    doctors_data = []

    # Extract data from each doctor info box
    for box in doctor_info_boxes:
        name = soup.find('h5', class_='nomargin').text.strip()
        specialty = soup.find('div', class_='f13 doc-specialty-name').text.strip()
        fees = soup.find('div', class_='fee').find_all('div')[1].text.strip()
        degree = soup.find('div', class_='degree d-none').find_all('div')[1].text.strip()
        
        timings_divs = box.find_all('div', class_='timings-col')
        timings = [
            {
                'time': div.find('div').text.strip(),
                'days': div.find('div', class_='f12').text.strip()
            }
            for div in timings_divs
        ]   
        
        doctor_data = {
            'name': name,
            'specialty': specialty,
            'fees': fees,
            'degree': degree,
            'timings': timings,
            'hospital': 'Memo Medical School Hospital'
        }
        
        doctors_data.append(doctor_data)
    
    path = "outputs/memon-medical-hospital.json"
    # Save the extracted data to a JSON file
    if os.path.exists(path):
        with open(path, 'r') as json_file:
            existing_data = json.load(json_file)
    else:
        existing_data = []

    # Append new data to existing data
    existing_data.extend(doctors_data)

    # Save back to the JSON file
    with open(path, 'w') as json_file:
        json.dump(existing_data, json_file, indent=4)


def read_urls_from_file(file_path):
    with open(file_path, 'r') as file:
        urls = [line.strip() for line in file.readlines()]
        print(urls)
    return urls
urls=read_urls_from_file('memon-medical.txt')
for url in urls:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    code(soup)

['https://mmi.edu.pk/find-a-doctor/?search-type=specialty&sp=75', 'https://mmi.edu.pk/find-a-doctor/?search-type=specialty&sp=73']


In [12]:
from bs4 import BeautifulSoup

html = '''
<div class="col-md-6 doctor-info-box active"> 
    <div class="position-relative doctor-info-box-in">  
        <div class="row-1 flexbox align-items-center "> 
            <div class="mr-10 thumb lazyloaded"> 
                <div style="background-image: url(&quot;data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAAAICTAEAOw==&quot;), url(&quot;https://cdn-fkafl.nitrocdn.com/SmcWMfFNujZDzRWxSfczwRCYPPFFCBMB/assets/images/optimized/rev-29316de/mmi.edu.pk/wp-content/uploads/2020/01/Dr.Naseer-Ahmed-Sheikh-1.jpg&quot;);" class="thumb-box bg-position-top bg-cover bg-norepeat lazyloaded" nitro-lazy-bg="https://cdn-fkafl.nitrocdn.com/SmcWMfFNujZDzRWxSfczwRCYPPFFCBMB/assets/images/optimized/rev-29316de/mmi.edu.pk/wp-content/uploads/2020/01/Dr.Naseer-Ahmed-Sheikh-1.jpg"></div> 
            </div>   
            <div class=""> 
                <a target="_new" href="https://mmi.edu.pk/book-appointment?d_id=4135"> 
                    <h5 class="nomargin">Dr. Naseer Ahmed</h5> 
                </a> 
                <div class="f13 doc-specialty-name"> NEUROLOGIST/NEURO PHYSICIAN </div> 
            </div> 
        </div> 
        <div class="col-md-12 content"> 
            <div class="content-box">  
                <div class="fee"> 
                    <div class="col-md-3" style="padding-left:0;font-weight:bold;">Fees</div> 
                    <div class="col-md-9" style="padding:0;">Rs 1700 </div> 
                </div> 
                <div class="flexbox flex-center jc-space-between"> 
                    <div class="f13 cursor-pointer basecolor1 timing-btn flexbox flex-center"> 
                        <div class="mr-10 semibold">See timings</div> 
                        <i class="fa fa-plus-circle"></i> 
                        <i class="fa fa-minus-circle"></i> 
                    </div> 
                    <a target="_new" href="https://mmi.edu.pk/book-appointment?d_id=4135" class="btn-border btn btn-sm btn-primary flexbox jc-center flex-center lazyloaded"> 
                        <div class="mr-10">Book Now</div> 
                        <i class="fa fa-arrow-circle-right"></i> 
                    </a> 
                </div> 
                <div class="timings">  
                    <div class="degree d-none"> 
                        <div class="col-md-3" style="padding-left:0;font-weight:bold;"> Degree </div> 
                        <div class="col-md-9" style="padding:0;"> MBBS, FCPS, Fellowship (Stroke) </div> 
                    </div> 
                    <div class="timings"> 
                        <div class="flexbox flex-center mb-1"> 
                            <i class="fa fa-clock-o mr-10"></i> 
                            <div class="font-weight-semibold">Timings</div> 
                        </div> 
                        <div class="row timings-row position-relative nomargin"> 
                            <div class="timings-col col-md-6 col-lg-6 col-6">
                                <div>09:00 AM to 01:00 PM</div>
                                <div class="f12">(Mon<span>, </span>Tue<span>, </span>Wed<span>, </span>Thu<span>, </span>Sat)</div>
                            </div>
                            <div class="timings-col col-md-6 col-lg-6 col-6">
                                <div>09:00 AM to 12:00 PM</div>
                                <div class="f12">(Fri)</div>
                            </div> 
                        </div> 
                    </div> 
                </div> 
            </div> 
        </div>   
    </div> 
</div>
'''

soup = BeautifulSoup(html, 'html.parser')

# Extract the doctor's name
doctor_name = soup.find('h5', class_='nomargin').text.strip()

# Extract the specialty
specialty = soup.find('div', class_='f13 doc-specialty-name').text.strip()

# Extract the fees
fees = soup.find('div', class_='fee').find_all('div')[1].text.strip()

# Extract the timings
timings_divs = soup.find_all('div', class_='timings-col')
timings = {}
for div in timings_divs:
    time = div.find('div').text.strip()
    days = div.find('div', class_='f12').text.strip()
    timings[days] = time

# Extract the degree
degree = soup.find('div', class_='degree d-none').find_all('div')[1].text.strip()

# Create a dictionary with the extracted information
doctor_info = {
    'name': doctor_name,
    'specialty': specialty,
    'fees': fees,
    'degree': degree,
    'timings': timings
}

print(doctor_info)


{'name': 'Dr. Naseer Ahmed', 'specialty': 'NEUROLOGIST/NEURO PHYSICIAN', 'fees': 'Rs 1700', 'degree': 'MBBS, FCPS, Fellowship (Stroke)', 'timings': {'(Mon, Tue, Wed, Thu, Sat)': '09:00 AM to 01:00 PM', '(Fri)': '09:00 AM to 12:00 PM'}}


# this web site is not allowing now to scrap data 

```
Bot Protection Firewall

Blocked because of Malicious Activities

Reference ID: 132091155366e3e7f8c5d2e
 
```

In [2]:
import json
from bs4 import BeautifulSoup
import requests
import os

def code(soup):
    # Find all doctor info boxes
    doctor_info_boxes = soup.find_all('div', class_='doctor-info-box-in')

    doctors_data = []

    # Extract data from each doctor info box
    for box in doctor_info_boxes:
        # Extract data
        name = box.find('h5', class_='nomargin').text.strip()
        specialty = box.find('div', class_='f13 doc-specialty-name').text.strip()
        fees = box.find('div', class_='fee').find_all('div')[1].text.strip()
        degree = box.find('div', class_='degree d-none').find_all('div')[1].text.strip()

        # Extract timings
        timings_divs = box.find_all('div', class_='timings-col')
        timings = [
            {
                'time': div.find('div').text.strip(),
                'days': div.find('div', class_='f12').text.strip()
            }
            for div in timings_divs
        ]

        # Create a dictionary for the doctor's information
        doctor_data = {
            'name': name,
            'specialty': specialty,
            'fees': fees,
            'degree': degree,
            'timings': timings,
            'hospital': 'Memon Medical School Hospital'
        }

        # Add to list
        doctors_data.append(doctor_data)
    
    # Define output path
    path = "outputs/memon-medical-hospital-dumy.json"

    # Save the extracted data to a JSON file
    if os.path.exists(path):
        with open(path, 'r') as json_file:
            existing_data = json.load(json_file)
    else:
        existing_data = []

    # Append new data to existing data
    existing_data.extend(doctors_data)

    # Save back to the JSON file
    with open(path, 'w') as json_file:
        json.dump(existing_data, json_file, indent=4)

def read_urls_from_file(file_path):
    with open(file_path, 'r') as file:
        urls = [line.strip() for line in file.readlines()]
    return urls

# Read URLs from file and process each URL
urls = read_urls_from_file('memon-medical.txt')
for url in urls:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    code(soup)
